<a href="https://colab.research.google.com/github/SocialQu/Branding/blob/main/Decision_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests

In [123]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

In [54]:
from google.colab import files
uploaded = files.upload()

In [105]:
df = pd.read_csv("tweets.csv")

In [177]:
def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [228]:
train_ds_pd, test_ds_pd = split_dataset(df, 0)


In [229]:
train_ds_pd.shape

(1867, 24)

In [230]:
label = "ENGAGEMENTS"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
full_ds = tfdf.keras.pd_dataframe_to_tf_dataset(df, label=label, task=tfdf.keras.Task.REGRESSION)


In [231]:
# Configure the model.
model_7 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

# Optional.
model_7.compile(metrics=["mse"])

# Train the model.
with sys_pipes():
  model_7.fit(x=train_ds)


Use /tmp/tmpj_saec61 as temporary training directory
Starting reading the dataset
1/2 [==============>...............] - ETA: 0s
Dataset read in 0:00:00.395992
Training model
Model trained in 0:00:02.433378
Compiling model
2/2 [==============================] - 3s 3s/step


In [232]:
# Evaluate the model on the test dataset.
evaluation = model_7.evaluate(train_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")

2/2 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - mse: 5226.8569
{'loss': 0.0, 'mse': 5226.85693359375}

MSE: 5226.85693359375
RMSE: 72.29700501122954


In [233]:
evaluation = model_7.evaluate(test_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:921: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: ignored

In [234]:
evaluation = model_7.evaluate(full_ds, return_dict=True)

print(evaluation)
print()
print(f"MSE: {evaluation['mse']}")
print(f"RMSE: {math.sqrt(evaluation['mse'])}")

2/2 [==============================] - 0s 43ms/step - loss: 0.0000e+00 - mse: 5226.8569
{'loss': 0.0, 'mse': 5226.85693359375}

MSE: 5226.85693359375
RMSE: 72.29700501122954


In [235]:
predictions = model_7.predict(full_ds)

In [236]:
np.savetxt("predictions.csv", predictions, delimiter=",")

,DAY,HOUR,MEDIA,LINKS,EMOJIS,LENGTH,HASHTAGS,MENTIONS,FOLLOWERS,FOLLOWING,LINEBREAKS,WE_1,WE_2,WE_3,WE_4,WE_5,WE_6,WE_7,WE_8,WE_9,WE_10,WE_11,WE_12,ENGAGEMENTS
0,0.285714,0.666667,-0.231542,-0.749093,-0.603294,-0.932305,-0.200735,-0.911891,-0.326671,0.630855,-0.494417,-0.085075,0.048697,0.365757,0.289738,-0.159383,-0.022961,-0.213818,0.090838,0.042079,-0.017159,0.02549,0.010372,0


In [165]:
predictions_2 = model_7.predict(predict_ds)

In [166]:
predictions_2

array([[1.8971311]], dtype=float32)

In [167]:
test_predictions = model_7.predict(train_ds)
train_predictions = model_7.predict(test_ds)


In [168]:
len(test_predictions)

187

In [169]:
len(train_predictions)

187